---
title: "Hosting a static Blog on Bare-Metals Kubernetes - This could have been a GitHub pages site…"
author: "Sarem"
date: "2024-10-01"
categories: [Kubernetes]
format:
  html:
    toc: true
    toc-location: left
    toc-depth: 2
    toc-title: Contents
description-meta: "Hosting a static Blog on Bare-Metals Kubernetes"
---

## Introduction
In today’s tech landscape, Kubernetes has become synonymous with scalable and resilient application hosting. But what happens when you combine it with the relatively humble task of hosting a static blog? Well, I went down that rabbit hole and set up this blog on a bare-metal Kubernetes cluster. To be exact, this is the second time I have done this. After switching back and forth between various content management systems for blogs, I have decided it was finally time to move back to a static [quarto](https://quarto.org/) site, hosted on good old K8s.

Could I have done this with GitHub Pages? Sure. Did I want to take the long (and more complicated) route? Absolutely.

Let me walk you through my experience setting up a Kubernetes-powered blog, and how it all came together.

## Design Goals
From the outset, I had some design goals for this project:

1. **Cost-effectiveness**: I wanted the whole setup to be cheap. No managed Kubernetes, no fancy cloud providers — just affordable VMs running the underlying Kubernetes nodes.
2. **Continuous deployment**: Any push to the blog's master branch in GitHub should automatically deploy the latest blog content.
3. **Notebooks integration**: There also exists [another repository](https://github.com/SaremS/sample_notebooks) where I store rather raw, uncommented experiments in Jupyter notebooks, too. Despite them being mostly code-only, I found that they might still be interesting to read so they should be included here as well. Thus, each deployment of the main blog should also copy and include all notebooks from this second repository. 
4. **Simplicity**: Although the underlying infrastructure is rather complex, deploying and updating the blog should be rather simple. This means, ideally, no crazy CRD extensions but only out-of-the-box Kubernetes resources where possible.

## Kubernetes Cluster Setup

### Node Setup
Since managed Kubernetes was out of the question, I opted for a bare-metal Kubernetes setup with VMs. I rented three virtual machines from [Contabo](https://contabo.com/), a fairly budget-friendly hosting provider (no affiliation on my end). While they are not the most reliable VM provider and spinning up a new VM takes quite some time, they are cheap and things are working nicely most of the time.

With three machine as cluster nodes, I created one master and two worker nodes. This obviously doesn't ensure high availability at all but keep in mind that we are still talking about a personal blog and not a critical production environment. 

To automate some initial configuration, I created an [Ansible](https://www.ansible.com/) playbook. The primary advantage here is being able to quickly tear down and rebuild the entire setup if necessary. As managing all necessary firewall configurations between the VMs turned out to be quite tedious, I also used [WireGuard](https://www.wireguard.com/) to set up a virtual network overlay. On the one hand, this adds some additional communication overhead. On the other hand, this only left the WireGuard port to worry about regarding communication amongst the cluster nodes.


### Cluster Setup
For the Kubernetes cluster, I initially tried a raw *kubeadm* setup, but configuring [Cilium](https://cilium.io/) as the CNI turned out to be much harder than expected. Accepting my defeat (for now), I went with Rancher's [RKE2](https://docs.rke2.io/) and things went relatively smooth from there. Installing [ArgoCD](https://argo-cd.readthedocs.io/en/stable/) then completed the, more or less, manual parts of the set up. Every other installation is now managed by Argo. 

Although persistent storage is not necessary at this point, I did some experiments that required setting up PVCs. As this is already an RKE2 cluster, it made sense to use [Longhorn](https://longhorn.io/) which was also reasonably straightforward to install. 

Finally, to enable external traffic, I added [metallb](https://metallb.universe.tf/) to the Kubernetes cluster and installed [nginx](https://nginx.org/en/) to the control plane VM. Nginx then accepts external traffic and performs a proxy pass to the exposed metallb services. 

![Nginx acts as a reverse proxy and passes external traffic to the corresponding metallb service](/images/hosting-a-static-blog-on-kubernetes/nginx-metallb.svc)

You can find all cluster-wide ArgoCD installations [here](https://github.com/SaremS/kubernetes-deployments/tree/master/cluster).

## Hosting the Blog on Kubernetes
Now that the cluster was up and running, it was time to host the actual blog. Remember that an important goal is to keep the manual effort for deploying new edits at a minimum. [As described by the quarto docs](https://quarto.org/docs/publishing/github-pages.html#github-action), the probably easiest way to host and deploy quarto sites is via [GitHub](https://pages.github.com/). This is unfortunately out of the question, since I will likely add some non-static things in the future anyway. T

### Quarto Docker Container
I’m using a Quarto Docker container to build and serve the blog. On each push to the master branch, the blog content is automatically built. What’s more, the Docker container pulls Jupyter notebooks from another repository (sample_notebooks) and incorporates them into the blog.

To keep things in sync, a push to sample_notebooks also triggers a pipeline that ensures the latest notebooks are included in the blog.

### Serving Static Files via Caddy
I chose Caddy to serve the blog’s static files. Why not Nginx? Well, Caddy has a simpler configuration and automatically obtains TLS certificates from Let’s Encrypt, which saved me time and hassle.

### Automatic Deployments with Argo Image Updater
One of the key goals of this setup was to have fully automated deployments. Whenever the master branch is updated, the latest version of the blog should automatically go live. Argo Image Updater made this possible. It watches for new Docker images and updates the blog deployment accordingly. Although the documentation warns that it isn’t production-ready, it’s been working flawlessly for me so far.

To use Argo Image Updater, I needed to deploy ArgoCD using Helm or Kustomize (a raw Kubernetes manifest wouldn’t cut it).

### Customizable Helm Chart
To facilitate continuous deployment, I used a customizable Helm chart. This chart allows me to tweak settings, like configuring the blog’s persistent storage and setting the target IP for the load balancer. There’s room to template even more configurations, but I wanted to keep things simple.

## Some Key Takeaways

RKE2 makes Kubernetes setup a breeze compared to kubeadm. I’ve previously set up Kubernetes clusters, but using RKE2 with Cilium has been a much smoother experience.

### Stability
So far, the cluster has been very stable, with no unexpected outages (except those caused by my own misconfigurations). However, the workload is minimal—serving a static blog isn’t exactly high traffic.

### Future Plans
I plan to add small, interactive in-notebook applications as showcases, with the app logic also running inside Kubernetes. I’ve done something similar in a previous version of my blog, but this setup is far cleaner.

## Conclusion
Could I have hosted this blog on GitHub Pages? Absolutely. But where’s the fun in that? Hosting a static blog on bare-metal Kubernetes might be overkill, but it has taught me a lot about automating deployments, integrating notebooks, and managing a Kubernetes cluster for low-traffic workloads.

Kubernetes isn’t just for large-scale applications. If you’re looking for a fun project to test your Kubernetes skills, hosting a blog can be an entertaining (and educational) challenge. Plus, it’s a great excuse to experiment with tools like RKE2, Caddy, ArgoCD, and Longhorn in a practical setting.

So, next time you think about hosting a static blog, consider Kubernetes. It might be over-engineering, but it’s a fantastic way to learn more about this powerful platform.